# 🤖 Tutorial 4: ReAct Agent - Autonomous Compliance Verification

## 🎯 What You'll Learn

In this tutorial, you'll see how all the components from the previous tutorials work together in a **ReAct Agent** that can:

1. **🧠 Reason** about building compliance problems autonomously
2. **🔧 Use tools** to access data, perform calculations, and query regulations
3. **📚 Query documents** using the RAG system from Tutorial 3
4. **🏗️ Analyze buildings** using calculations from Tutorial 2
5. **💬 Have conversations** about compliance in natural language

## 📋 Table of Contents

1. [What is a ReAct Agent?](#1-what-is-a-react-agent)
2. [Setting Up the Agent](#2-setting-up-the-agent)
3. [Understanding the Tools](#3-understanding-the-tools)
4. [Testing Simple Queries](#4-testing-simple-queries)
5. [Complex Compliance Verification](#5-complex-compliance-verification)
6. [Conversational Interface](#6-conversational-interface)

---


## 1. What is a ReAct Agent?

**ReAct** = **Rea**soning + **Act**ing

A ReAct agent is an AI system that can:
1. 🧠 **Think** about what it needs to do
2. 🔧 **Choose** which tools to use
3. ⚡ **Execute** the tools
4. 📊 **Observe** the results
5. 🔁 **Repeat** until it has the answer

### How it works:

```
User: "Check if all doors meet requirements"
     ↓
Agent: "I need to list all doors first"
     ↓
Tool: list_all_doors()
     ↓
Agent: "Now I'll check each door's compliance"
     ↓
Tool: check_door_width_compliance("D001")
     ↓
Agent: "Let me query the regulations for requirements"
     ↓
Tool: query_normativa("minimum door width")
     ↓
Agent: "Here's my final answer..."
```

The agent **autonomously** decides what to do and uses the right tools to get the information it needs!


## 2. Setting Up the Agent

Let's set up our ReAct agent using the components we've already built!


In [4]:
# Import required libraries
import sys
from pathlib import Path
from langchain_core.messages import HumanMessage

# Add src to path for imports
sys.path.append('.')

# Import our existing agent components
from src.agent.graph import create_compliance_agent
from src.agent.tools import load_project_data, set_vectorstore_manager

print("✅ Libraries imported successfully!")
print("🤖 Ready to use our ReAct agent")


ModuleNotFoundError: No module named 'src'

In [5]:
# Load project data (use example dataset from Tutorial 1/2)
project_file = Path("data/extracted/tutorial_example.json")
load_project_data(project_file)
print(f"✅ Project data loaded: {project_file}")

# Setup RAG for query_normativa tool if available
rag_available = False
vectorstore_path = Path("vectorstore/normativa_db")
try:
    from src.rag.vectorstore_manager import VectorstoreManager
    from src.rag.qa_chain import create_qa_chain
    rag_available = vectorstore_path.exists() and any(vectorstore_path.iterdir())
except Exception:
    rag_available = False

if rag_available:
    try:
        rag = VectorstoreManager(vectorstore_path)
        rag.load_existing()
        retriever = rag.get_retriever(k=3)
        qa_chain = create_qa_chain(retriever)

        class RAGWrapper:
            def query(self, question):
                return qa_chain({"query": question})

        set_vectorstore_manager(RAGWrapper())
        print("✅ RAG system connected")
    except Exception as e:
        print("⚠️ Could not initialize RAG:", e)
        print("   Tip: Create the vectorstore before the demo.")
        print("   For setup, see docs or run: python scripts/create_vectorstore.py")
else:
    print("ℹ️ RAG vectorstore not found. Skipping CTE querying in this run.")
    print("   To enable, run: python scripts/create_vectorstore.py")

# Create agent
agent = create_compliance_agent()
print("✅ Agent created")
print("\n🤖 Ready to verify compliance!")


NameError: name 'load_project_data' is not defined

## 3. Understanding the Tools

Our ReAct agent has access to 6 powerful tools that combine all the functionality from the previous tutorials:

### 🏗️ **Data Tools** (from Tutorial 1)
- **`get_room_info(room_id)`** - Get detailed room information
- **`get_door_info(door_id)`** - Get detailed door information  
- **`list_all_doors()`** - List all doors in the project

### 📐 **Calculation Tools** (from Tutorial 2)
- **`check_door_width_compliance(door_id)`** - Check door width compliance
- **`calculate_egress_distance(room_id)`** - Calculate evacuation distances

### 📚 **RAG Tool** (from Tutorial 3)
- **`query_normativa(question)`** - Query Spanish building codes (CTE)

The agent can use these tools **autonomously** to answer complex compliance questions!


## 4. Testing Simple Queries

Let's start with some simple queries to see the agent in action!


In [6]:
# Example 1: List all doors
print("="*70)
print("EXAMPLE 1: List all doors")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="List all doors in the project")],
    "iterations": 0
})

# Print agent's response
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)

print(f"\n✅ Completed in {result['iterations']} iterations")


EXAMPLE 1: List all doors



NameError: name 'agent' is not defined

In [7]:
# Example 2: Check door compliance
print("="*70)
print("EXAMPLE 2: Check door compliance")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="Check if all doors meet minimum width requirements")],
    "iterations": 0
})

# Print agent's reasoning and answer
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


EXAMPLE 2: Check door compliance



NameError: name 'agent' is not defined

## 5. Complex Compliance Verification

Now let's see the agent handle more complex tasks that require multiple tools and reasoning!


In [8]:
# Example 3: Complex compliance check
print("="*70)
print("EXAMPLE 3: Complex Compliance Check")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="""
    Perform a comprehensive compliance check:
    1. List all doors
    2. Check each door's width compliance
    3. Calculate evacuation distances from all rooms
    4. Summarize findings with specific non-compliant items
    """)],
    "iterations": 0
})

for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


EXAMPLE 3: Complex Compliance Check



NameError: name 'agent' is not defined

## 6. Conversational Interface

Finally, let's create a simple conversational interface where you can ask the agent anything!


In [9]:
# Create a conversational interface function
def chat_with_agent(question: str) -> str:
    """
    Chat with the ReAct agent.
    
    Args:
        question: The question to ask the agent
    
    Returns:
        String with the agent's response
    """
    print(f"\n💬 Question: {question}")
    print("=" * 80)
    
    # Create initial state
    initial_state = {
        "messages": [HumanMessage(content=question)],
        "iterations": 0
    }
    
    # Run the agent
    result = agent.invoke(initial_state)
    
    # Extract the final response
    final_response = ""
    for message in result["messages"]:
        if hasattr(message, 'content') and message.content and not message.content.startswith("You are"):
            final_response = message.content
    
    print(f"\n🤖 Agent Response:")
    print(final_response)
    print(f"\n📊 Completed in {result['iterations']} iterations")
    
    return final_response

print("✅ Conversational interface created")
print("💬 Use chat_with_agent('your question') to interact")


✅ Conversational interface created
💬 Use chat_with_agent('your question') to interact


In [10]:
# Demo the conversational interface
print("🎭 DEMO: Conversational Interface")
print("=" * 50)

# Example conversations
demo_questions = [
    "What building codes apply to emergency exit doors?",
    "Tell me about the largest room in the project",
    "Check if all doors in the project meet compliance requirements",
    "Analyze the building's circulation and connectivity"
]

for i, question in enumerate(demo_questions, 1):
    print(f"\n{'='*20} DEMO {i} {'='*20}")
    response = chat_with_agent(question)
    
    if i < len(demo_questions):
        input("\nPress Enter to continue to next demo...")

print("\n🎉 Demo completed!")
print("\n💡 Try your own questions:")
print("   chat_with_agent('Your question here')")


🎭 DEMO: Conversational Interface

==================== DEMO 1 ====================

💬 Question: What building codes apply to emergency exit doors?


NameError: name 'agent' is not defined

## 🎯 Summary

Congratulations! You've seen how all the components from the previous tutorials work together in a powerful ReAct agent:

### 🏗️ **What We Built**

1. **ReAct Agent**: Autonomous reasoning and acting system
2. **6 Specialized Tools**: Combining data access, calculations, and regulations
3. **Natural Language Interface**: Conversational compliance verification
4. **Complete Integration**: All tutorials working together seamlessly

### 🛠️ **Available Tools**

**Building Data** (from Tutorial 1):
- `get_room_info(room_id)` - Get detailed room information
- `get_door_info(door_id)` - Get detailed door information  
- `list_all_doors()` - List all doors in project

**Calculations** (from Tutorial 2):
- `check_door_width_compliance(door_id)` - Check door compliance
- `calculate_egress_distance(room_id)` - Calculate evacuation distances

**Regulations** (from Tutorial 3):
- `query_normativa(question)` - Query building codes with RAG

### 🚀 **Key Features**

- **Autonomous Reasoning**: Agent decides which tools to use
- **Multi-step Analysis**: Can combine multiple tools for complex tasks
- **Natural Language**: Conversational interface
- **Comprehensive**: Covers data, calculations, and regulations
- **Extensible**: Easy to add new tools and capabilities

### 💡 **Next Steps**

You can now:
1. **Ask Complex Questions**: The agent handles multi-step compliance checks
2. **Add More Tools**: Create tools for specific building types or regulations
3. **Integrate with CAD**: Connect to real CAD software
4. **Deploy**: Create a web interface or API

**This is the future of building compliance verification!** 🎉

### 🎓 **What You Learned**

- How ReAct agents work (Reasoning + Acting)
- How to use existing tools with LLMs
- How to build agent workflows with LangGraph
- How to integrate RAG for document querying
- How to create conversational AI interfaces

**The agent can now autonomously verify building compliance using all the tools we've built!**
